In [ ]:
from rouge_score.rouge_scorer import _create_ngrams
from nltk.stem.porter import PorterStemmer
import spacy, six, json, openai
from utils import tokenize
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

PorterStemmer = PorterStemmer()
nlp = spacy.load('en_core_web_sm')

In [ ]:

def plot_roc_curve(human_scores, gpt_scores):
    # Data
    A = human_scores
    B = gpt_scores
    # Combine scores and true labels
    scores = A + B
    labels = [0] * len(A) + [1] * len(B)
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(labels, scores)
    # Calculate AUC (Area Under Curve)
    roc_auc = auc(fpr, tpr)
    # Plot ROC curve
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve: Open-gen w/ GPT3.5-Reddit w prompts' )
    plt.legend(loc="lower right")
    plt.show()
    # what is the TPR for FPR = 0.1?
    for idx, fpr_ in enumerate(fpr):
        if fpr_ > 0.01:
            print(f"TPR at 1% FPR: {tpr[idx]:.4f}")
            break

In [ ]:
with open('results/regen_davinci003_20_0.7.jsonl', "r") as f:
    regen_davinci003_20_05 = [json.loads(x) for x in f.read().strip().split("\n") ]
len(regen_davinci003_20_05)

In [ ]:
def get_ratio_avgk(instance, num_samples=20):
    truncate_len = len( instance['original_human_response_truncate']['choices'][0]['logprobs']['token_logprobs'] )
    orignal_prob = instance['original_human_response']['choices'][0]['logprobs']['token_logprobs'][truncate_len:]
    orignal_logprob = np.mean(orignal_prob) # / len(orignal_prob)
    regen_probs = [ sum( instance['gold_gen_regen']['choices'][i]['logprobs']['token_logprobs']) / \
                    len( instance['gold_gen_regen']['choices'][i]['logprobs']['token_logprobs'] ) for i in range(num_samples) \
                    if len( instance['gold_gen_regen']['choices'][i]['logprobs']['token_logprobs'] ) != 0 ]
    regen_logprobs_avg_20 = np.mean(regen_probs)  
    original_th = orignal_logprob - regen_logprobs_avg_20

    truncate_gen_len = len( instance['original_gen_response_truncate']['choices'][0]['logprobs']['token_logprobs'] )
    gen_prob = instance['original_gen_response']['choices'][0]['logprobs']['token_logprobs'][truncate_gen_len:]
    gen_logprob = np.mean(gen_prob) # 
    gen_regen_probs = [ sum( instance['gen_completion_regen']['choices'][i]['logprobs']['token_logprobs']) / \
                    len( instance['gen_completion_regen']['choices'][i]['logprobs']['token_logprobs'] ) for i in range(num_samples) \
                        if len( instance['gen_completion_regen']['choices'][i]['logprobs']['token_logprobs'] ) != 0 ]
    gen_regen_logprobs_avg_20 = np.mean(gen_regen_probs) 
    gen_th = gen_logprob - gen_regen_logprobs_avg_20

    return original_th, gen_th

In [ ]:
########################## with questions ##########################
human_scores, gpt_scores = [], []
for idx, instance in enumerate( regen_davinci003_20_05 ):
    original_th, gen_th = get_ratio_avgk(instance, num_samples = 20)
    human_scores.append( original_th )
    gpt_scores.append( gen_th )

In [ ]:
# remove nan
human_scores = [x for x in human_scores if str(x) != 'nan']
gpt_scores = [x for x in gpt_scores if str(x) != 'nan']

In [ ]:
# plot and give different colors
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot( human_scores , label='human')
plt.plot(  gpt_scores, label='gpt')
plt.legend()
plt.show()

In [ ]:
plot_roc_curve( human_scores, gpt_scores )